# Пошаговое описание работы решения:
Этап 1: Подготовка сервера с языковой моделью

- Запуск модели: С помощью фреймворка vllm с hugging face разворачивается специализированная модель для логических рассуждений с использованием инструментов Qwen3-4B-Thinking-2507-FP8 на локальном сервере.

Этап 2: Инициализация агента

- Настройка подключения: Создается конфигурация для подключения к запущенному локальному серверу.

- Создание агента: Инициализируется Assistant из библиотеки qwen-agent, которому передается доступ к инструменту code_interpreter (интерпретатор кода Python). Это позволяет агенту выполнять точные вычисления и решать задачи не только аналитически, но и практическими методами.

Этап 3: Обработка задачи перед решением

Перед решением для каждой математической задачи выполняются следующие шаги:

  - Маскировка: Все математические выражения (внутри знаков \$, например, \$\frac{a}{b}\$) заменяются на временные токены ( \_\_MATH\_0\_\_), чтобы переводчик их не исказил.

  - Перевод: Модели-агенту дается инструкция перевести текст задачи с русского на английский, сохраняя токены \_\_MATH\_0\_\_ неизменными.

  - Восстановление: После перевода математические выражения подставляются обратно на свои места, вместо токенов \_\_MATH\_0\_\_.

Этап 4: Генерация решения и ответа (Многократный запрос с Majority Voting):

В процессе решения происходит следующее:
- Агент получает системный промпт, который предписывает сочетать естественные рассуждения с выполнением программного кода для решения задачи и оформлять окончательный ответ внутри команды \boxed{}, а также инструкции по формату ответа для избежания путаниц и пояснения спорных случаев в формате ответа.

- Парсинг ответа: Функция extract_answer находит в тексте, сгенерированном моделью, все блоки \boxed{}, корректно обрабатывает вложенные скобки и объединяет содержимое нескольких блоков в строку.

- Повторение для надежности: Для одной и той же задачи решение генерируется от 2 до 7 раз. (Пока доля наиболее частого ответа ≤ 50%). Финальным ответом выбирается тот, который встречается чаще всего (Majority Voting). Это повышает надежность и точность системы.

Структура рабочей директории:

```
├── model-2.ipynb
└── data/
    └── input/                        # входные данные
        ├── final_submission.csv
        ├── submission.csv
        ├── test_private.csv
        ├── test_public.csv
        ├── train.csv
    └── working/2/                    # появляется во время решения задач
        └── full-solutions/           # полные логи решений
            ├── task-0.json
            ├── task-1.json
            ├── task-2.json
            ├── ...
        ├── solving.log               # краткие логи решений
        ├── vllm_server.log           # логи vllm-сервера
    └── submissions/
        ├── subimssion_private_2.csv  # файл с ответами появляется здесь
```

# Установка библиотек и зависимостей

In [ ]:
!pip install -U "qwen-agent[gui,rag,code_interpreter,mcp,python_executor]"
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mcp to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
from collections import Counter
import subprocess
import os
import requests
import time
from tqdm import tqdm

import re

from qwen_agent.agents import Assistant

import pandas as pd

from tqdm.notebook import tqdm as tqdmn

import json

In [ ]:
# отключаем предупреждения
import warnings
warnings.filterwarnings("ignore")

In [ ]:
root = Path('/content/drive/MyDrive/AIIJC-2025-LLM/')

In [ ]:
# создаем директории для логирования
(root / 'data/working/2/full-solutions').mkdir(parents=True, exist_ok=True)

# Создание и вызов агента

In [ ]:
def get_cmd():
  # с помощью vllm разворачиваем на сервере модель Qwen3-4B-Thinking-2507-FP8, по hugging face ссылке на модель, фиксируем seed для воспроизводимости
    return (
    'vllm serve Qwen/Qwen3-4B-Thinking-2507-FP8 --port 8000 --max-model-len 32000 --tensor-parallel-size 1 --seed 3407 --reasoning-parser deepseek_r1')

def wait_for_server(url='http://0.0.0.0:8000', timeout=300, interval=1):
    start_time = time.time()
    while True: # подключаемся к серверу
        try:
            response = requests.put(url)
            if response.status_code != 403: # проверяем, отвечает ли сервер
                return True
        except requests.RequestException:
            if time.time() - start_time > timeout: # ограничиваем время на одну попытку подключения к серверу
                raise TimeoutError("Server did not respond within timeout period")
            time.sleep(interval)

In [ ]:
cmd = get_cmd()
print(cmd)
log_path = Path(root / "data/working/2/vllm_server.log").resolve() # создаем файл для логирования действий на сервере
log_file = open(log_path, "w", buffering=1)
proc = subprocess.Popen(cmd, shell=True, stdout=log_file, stderr=subprocess.STDOUT, preexec_fn=os.setsid) # отправляем модель на сервер
wait_for_server()
print(f"Server is ready!")

vllm serve Qwen/Qwen3-4B-Thinking-2507-FP8 --port 8000 --max-model-len 32000 --tensor-parallel-size 1 --seed 3407 --reasoning-parser deepseek_r1
Server is ready!


In [ ]:
llm_cfg = {
    'model': 'Qwen/Qwen3-4B-Thinking-2507-FP8',
    'model_server': 'http://localhost:8000/v1',
    'api_key': 'EMPTY',
    'generate_cfg': {
        'temperature': 0.6,
        'seed': 3407
    },
}

# создаем агента на базе модели, задаем список инструментов, которые он может использовать
bot = Assistant(llm=llm_cfg, function_list=['code_interpreter'])

# Вспомогательные функции

In [ ]:
def extract_answer(generated_text):
    # модель пишет финальный ответ после фразы "Final Answer", ищем ее последнее вхождение
    final_answer_pos = generated_text.rfind("Final Answer")

    if final_answer_pos != -1:
        search_text = generated_text[final_answer_pos:]
    else:
        search_text = generated_text

    # объединяем информацию из нескольких \boxed{}
    boxes = []
    current_pos = 0

    while True:
        start_idx = search_text.find('\\boxed{', current_pos)
        if start_idx == -1:
            break

        start_idx += len('\\boxed{')
        balance = 1
        end_idx = start_idx

        while end_idx < len(search_text) and balance > 0:
            if search_text[end_idx] == '{':
                balance += 1
            elif search_text[end_idx] == '}':
                balance -= 1
            end_idx += 1

        if balance == 0:
            box_content = search_text[start_idx:end_idx-1].replace(',', ';')
            boxes.append(box_content)

        current_pos = end_idx

    return ('[' + ';'.join(boxes) + ']') if boxes else 'PARSING_ERROR'

In [ ]:
def translate(text):
    # маскируем важные математические символы и выражения, чтобы не исказить их при переводе
    math_expressions = []
    def mask_math(match):
        math_expressions.append(match.group(0))
        return f"__MATH_{len(math_expressions)-1}__"

    masked_text = re.sub(r'\$.*?\$', mask_math, text)

    # переводим текст
    prompt = "Translate this math problem from Russian to English, keeping all __MATH_X__ placeholders unchanged. also replace repeating sentences with just one:" + masked_text
    messages = [
        {"role": "user", "content": prompt}
    ]
    for response in bot.run(messages=messages):
        pass
    translated_text = (response[-1]['content'])

    # возвращаем замаскированные математические выражения и символы в переведенное условие задачи
    def unmask_math(match):
        return math_expressions[int(match.group(1))]

    final_text = re.sub(r'__MATH_(\d+)__', unmask_math, translated_text)

    return final_text

# Загрузка датасета

In [ ]:
test_df = pd.read_csv(root / 'data/input/final_submission.csv')

In [ ]:
test_df

,task,answer
0,Вычислить $\displaystyle \lim_{n \to \infty} \...,[0]
1,Вычислить $\displaystyle \lim_{n \to \infty} (...,[0]
2,Вычислить интеграл: $\displaystyle \int_{0}^{\...,[0]
3,Два охотника стреляют по волку (вероятности по...,[0]
4,Вычисли $ \displaystyle \int_1^2 \dfrac{2e^{2x...,[0]
...,...,...
339,Вычислить $\displaystyle \lim_{n \to \infty} \...,[0]
340,Вычисли $ \displaystyle \lim_{n \rightarrow \i...,[0]
341,Для всех действительных $x$ и $y$ выполняется ...,[0]
342,Вычислить определитель: \[ \begin{vmatrix} -7 ...,[0]


# Функция получения и агрегации ответов модели

In [ ]:
global con
con =  {i: [] for i in range(len(test_df))}

In [ ]:
def solve(i,q):
    # переводим задачу на английский язык
    q = translate(q)
    print(q)
    answers = []
    print(i, end='')

    # генерируем ответы, пока не достигнут достаточный уровень уверенности в одном из них
    while (len(answers) < 2) or ((len(answers) >= 2) and (len(answers) < 7) and (((Counter(answers).most_common(1)[0][1] / len(answers)) <= 0.5) or (Counter(answers).most_common(1)[0][0] == 'PARSING_ERROR'))):
        # передаем модели переведенную задачу, промпт и инструкции по формату ответа модели во избежание путаниц
        messages =  [{'role': 'user', 'content':
                      'Please integrate natural language reasoning with programs to solve the problem below, and put your final answer within \\boxed{}. Use code_interpreter tool for precise calculations.\n\n' + \
                      q + \
                      '\n\nGeneral instructions:\n1. If the answer is /infty or -/infty put it as it is.\n2. Don\'t assume that there are mistakes in problem statement, it is absolutely correct.\n3. If the task is to solve the equation put each root in the answer only once regardless of their multiplicity.\n4. Use "." as the decimal delimiter.\n5. If the task is to calculate limit or integral then use code_interpreter tool for calculations (e.g. sympy) to avoid mistakes.\n6. If the task is to find out something like "how many times to do something to ..." then remember that the answer must be integer so it should be rounded up properly, e.g. you can\'t pick a ball 1.5 times.\n7. If you need to find the probability then carefully analyze boundary cases (e.g. be careful with questions like "find the probability that N balls are white", such questions should NOT be interpreted as "find the probability that AT LEAST N balls are white" but "find the probability that EXACTLY N balls are white AND NO MORE NOR LESS THAN N", use this principle for ALL similar problems).\n8. If the answer is a simple fraction and you are asked to put the sum of the numerator and the denominator, then don\'t put in the answer rounded fraction value, but put the required sum.\n9. If asked to find the number of something, it cannot be a fraction, but an integer.'}]
        # генерация решения
        ans = None
        t = time.time()
        for response in bot.run(messages=messages):
            if time.time() - t >= 10 * 60:
                ans = 'PARSING_ERROR'
                break
        # извлечение ответа из текста, сгенерированного моделью
        if ans is None:
            ans = extract_answer(response[-1]['content'])
        print(' || ' + ans, end='')
        answers.append(ans)
        con[i].append(response)

    c = Counter(answers).most_common() # majority voting - берем самый частый ответ
    # обработка случая, если самый частый ответ - ошибка парсинга
    if c[0][0] == 'PARSING_ERROR':
        if len(c) > 1:
            # если есть другие ответы, берем второй по популярности (не ошибку)
            return c[1][0], answers
        else:
            return 'PARSING_ERROR', answers
    else:
        return c[0][0], answers

In [ ]:
log_file = root / 'data/working/2/solving.log'

with open(log_file, 'w') as file:
    file.write('')

# Цикл решения задач

In [ ]:
t = time.time()
for i in tqdmn(range(len(test_df))):
    t1 = time.time()
    question = test_df.iloc[i].task
    ans, all_answers = solve(i, question)
    print()
    test_df['answer'].loc[i] = ans
    t1 = time.time() - t1
    with open(log_file, 'r+') as file:
        file.read()
        file.write(f'Problem {i} solved ({t1:.2f}s) || Final answer: "{test_df["answer"].loc[i]}" || All answers: {all_answers}\n')
    with open(root / f'data/working/2/full-solutions/task-{i}.json', 'w') as file:
        json.dump({
            'task': test_df.iloc[i].task,
            'answer': test_df["answer"].loc[i],
            'all_answers': all_answers,
            'time': t1,
            'raw': con[i]
        }, file)

  0%|          | 0/344 [00:00<?, ?it/s]



Calculate $\displaystyle \lim_{n \to \infty} \frac{n \cos(n^2 + e^n)}{2^n - 1}$ and write the answer as a decimal fraction rounded to three decimal places using mathematical rounding rules.
0

2025-09-22 20:26:23,576 - code_interpreter.py - 196 - INFO - INFO: kernel process's PID = 5383
INFO:qwen_agent_logger:INFO: kernel process's PID = 5383
2025-09-22 20:26:26,593 - code_interpreter.py - 128 - INFO - stdout:

```
Exception reporting mode: Minimal

```
INFO:qwen_agent_logger:stdout:

```
Exception reporting mode: Minimal

```


 || [0.000] || [0.000]


Calculate $\displaystyle \lim_{n \to \infty} (\sqrt{n + 1} - \sqrt{n})$ and write the answer as a decimal fraction, rounded to three decimal places using mathematical rounding rules.
1 || [0.000] || [0.000]


Compute the integral: $\displaystyle \int_{0}^{\pi} \sin 2x dx$. Write the answer as a decimal fraction, rounded to three decimal places according to mathematical rounding rules.
2 || [0.000] || [0.000]


Two hunters each fire one shot at a wolf (hit probabilities 0.7 and 0.8). Find the probability of at least one hit. Express the answer as a decimal rounded to three decimal places according to mathematical rounding rules.
3 || [0.940] || [0.940]


Compute $ \displaystyle \int_1^2 \dfrac{2e^{2x}-e^x}{\sqrt{3e^{2x}-6e^x-1}}dx$. Write the answer as a decimal fraction, rounding according to mathematical rounding rules to three decimal places.
4 || [6.620] || [6.620]


There are 8! eight-digit positive numbers that use each of the digits 1 through 8 exactly on

2025-09-23 06:52:10,240 - nous_fncall_prompt.py - 215 - WARNING - Invalid json tool-calling arguments
2025-09-23 06:52:10,249 - nous_fncall_prompt.py - 215 - WARNING - Invalid json tool-calling arguments
2025-09-23 06:52:10,263 - utils.py - 89 - WARNING - Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/qwen_agent/tools/code_interpreter.py", line 109, in call
    params = json5.loads(params)
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/json5/lib.py", line 205, in loads
    raise ValueError(err)
ValueError: <string>:1 Unexpected end of input at column 520

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/qwen_agent/utils/utils.py", line 291, in extract_code
    text = json5.loads(text)['code']
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/json5/lib.py", line 205, in loads
    raise Valu

 || [0.135]


Calculate $\displaystyle \lim_{n \to \infty} (\sqrt{n^2 + 2n + 2\sin(n + 2)} - \sqrt{n^2 + \sin(n - 3)})$ and write the answer as a decimal fraction, rounded to three decimal places according to mathematical rounding rules.
313 || [1.000] || [1.000]


Calculate $ \displaystyle \int_0^{+\infty} \ln\left(\dfrac{1+x^{\pi}}{1+x^e}\right) \cdot \dfrac{dx}{(1+x^2)\ln (x)}$ and write the answer as a decimal fraction rounded to three decimal places according to mathematical rounding rules.
314 || [0.332] || [0.332]


Calculate $\displaystyle \lim_{n \to \infty} \frac{\sqrt{n^2 + \sin(2^n + 1)}}{(\sqrt{n + 1})^2 + (\sqrt{n - 1})^2}$ and write the answer as a decimal fraction, rounded to three decimal places according to mathematical rounding rules.
315 || [0.500] || [0.500]


Calculate $\displaystyle \lim_{n \to \infty} \left(\frac{3n + 1}{4n + 5}\right)^n$ and write the answer as a decimal fraction, rounded to three decimal places according to mathematical rounding rules.
316 || 

In [ ]:
test_df

,task,answer
0,Вычислить $\displaystyle \lim_{n \to \infty} \...,[0.000]
1,Вычислить $\displaystyle \lim_{n \to \infty} (...,[0.000]
2,Вычислить интеграл: $\displaystyle \int_{0}^{\...,[0.000]
3,Два охотника стреляют по волку (вероятности по...,[0.940]
4,Вычисли $ \displaystyle \int_1^2 \dfrac{2e^{2x...,[6.620]
...,...,...
339,Вычислить $\displaystyle \lim_{n \to \infty} \...,[0.500]
340,Вычисли $ \displaystyle \lim_{n \rightarrow \i...,[0.000]
341,Для всех действительных $x$ и $y$ выполняется ...,[0.000]
342,Вычислить определитель: \[ \begin{vmatrix} -7 ...,[1.000]


In [ ]:
test_df.to_csv(root / 'data/submissions/subimssion_private_2.csv', index=False)